In [1]:
import pandas as pd
import seaborn as sns
from scipy import stats

In [2]:
df = pd.read_csv('../rawdata/Data03.csv')

In [3]:
df.head()

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,9.0,새마을금고,정상,43.0,1862.0
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,0,없음,2.0,현대카드,정상,62.0,2532.0
2,66756657,렌탈,Normal,홈쇼핑/방송,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,8.0,우리은행,정상,60.0,2363.0
3,66423450,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,계약확정,0,없음,5.0,농협은행,정상,60.0,2449.0
4,66423204,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,해약확정,12,있음,8.0,농협은행,해약,51.0,1942.0


## One Sample T Test (일 표본 평균 검정)

- 대립가설 : A값은 집단의 평균과 다르다
- 귀무가설 : A값은 집단의 평균과 같다

In [6]:
df['amount'].mean()

93994.98002105099

In [7]:
df['amount'].median()

96900.0

In [11]:
member_df = df[df['type_of_contract'] == '멤버십']

In [13]:
# 귀무가설 기각
stats.ttest_1samp(member_df['amount'], df['amount'].mean())

Ttest_1sampResult(statistic=-335.8142021618799, pvalue=0.0)

## Two Sample T Test (이표본 평균 검정)

- 두 집단의 평균의 차이가 있는지 검정
- 집단이 서로 등분산성을 보이지 않을 때 실시 (등분산성을 보인다면? -> 독립 Two Sample T Test)
- 독립 Two Sample T Test 코드 (stats.ttest_ind(df1['col1'], df2['col1'], equal_var =True))

- Levene Test : 두 집단 분산의 차이가 있는지 검정
    - 귀무가설 : 두 집단 분산이 같음
    - 대립가설 : 두 집단의 분산이 다름
    

In [15]:
df['type_of_contract'].value_counts()

렌탈     46481
멤버십     4819
Name: type_of_contract, dtype: int64

In [16]:
rental_df = df[df['type_of_contract'] == '렌탈']
membership_df = df[df['type_of_contract'] == '멤버십']

In [19]:
# 두 집단의 등분산 검정
# 귀무가설 : 렌탈 그룹과 멤버십 그룹의 amount값의 분산의 차이가 없음
# 대립가설 : 렌탈 그룹과 멤버십 그룹의 amount값의 분산의 차이가 있음
stats.levene(rental_df['amount'], membership_df['amount'])

LeveneResult(statistic=326.2155801732991, pvalue=1.0782290403372968e-72)

In [21]:
print(rental_df['amount'].var())
print(membership_df['amount'].var())

144928790.0446055
40851747.12434066


In [23]:
# 분산의 차이가 있는 걸로 나타났으므로 Two Sample T Test 진행
# 귀무가설 : 렌탈 그룹과 멤버십 그룹의 amount값의 차이가 없음
# 대립가설 : 렌탈 그룹과 멤버십 그룹의 amount값의 차이가 있음
stats.ttest_ind(rental_df['amount'], membership_df['amount'])

Ttest_indResult(statistic=193.95822270831934, pvalue=0.0)

In [24]:
print(rental_df['amount'].mean())
print(membership_df['amount'].mean())

97200.31561283105
63075.94625440963


## Chi Square Test

- 두 범주형 집단의 독립성 검정
- 귀무가설 : A 항목과 B항목은 서로 독립
- 대립가설 : A 항목과 B항목은 서로 종속

In [25]:
df['type_of_contract'].value_counts()

렌탈     46481
멤버십     4819
Name: type_of_contract, dtype: int64

In [26]:
df['product'].value_counts()

K1    39134
K2     8995
K3     2082
K5      645
K4      327
K6      120
Name: product, dtype: int64

In [29]:
# 계약유형에 따라 제품 선택이 달라지는지 검정
# Contingency Table - crosstab
crosstab_df = pd.crosstab(df['type_of_contract'], df['product'])

In [30]:
# 귀무가설 : 계약유형에 따라 제품군의 차이가 없음 (서로 독립)
# 대립가설 : 계약유형에 따라 제품군의 차이가 있음 (서로 종속)

# 첫번째 : Chi
# 두번째 : P-value
# 세번째 : degree of freedom
# Array : 기댓값
stats.chi2_contingency(crosstab_df)

(1340.8053403924034,
 9.218461328024673e-288,
 5,
 array([[3.54550553e+04, 8.15001462e+03, 1.88551200e+03, 2.96281799e+02,
         5.84409053e+02, 1.08727266e+02],
        [3.67594474e+03, 8.44985380e+02, 1.95488002e+02, 3.07182011e+01,
         6.05909472e+01, 1.12727344e+01]]))